In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import anndata as ad
import pandas as pd
from harmony import harmonize

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
sc.set_figure_params(fontsize = 20, dpi=50, dpi_save=50)

# Single Visium Analysis

In [ ]:
def prepare_for_umap(adata, resolution = 0.5, n_top_genes=2000, n_comps=50, batch_effects = False, df_metadata = None):
    sc.pp.normalize_total(adata, inplace=True)
    print('Finish normalization!')
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=n_top_genes)
    sc.pp.pca(adata, n_comps, use_highly_variable = True)
    print('Finish PCA!')
    
    if batch_effects:
        X_PCA = adata.obsm['X_pca'].copy()
        print("Processing shape: ", X_PCA.shape)
        corr_PCA = harmonize(X_PCA, df_metadata, batch_key = 'Sample ID')
        print("Finish correction!")
        adata.obsm['X_pca'] = corr_PCA
        
    sc.pp.neighbors(adata, use_rep = 'X_pca')
    sc.tl.umap(adata)
    sc.tl.leiden(adata, key_added="clusters", resolution = resolution)

In [ ]:
HCC_1N = './raw_data/Adjacent/HCC-1N'
HCC_2N = './raw_data/Adjacent/HCC-2N'
HCC_3N = './raw_data/Adjacent/HCC-3N'
HCC_4N = './raw_data/Adjacent/HCC-4N'

HCC_1L = './raw_data/Leading_Edge/HCC-1L'
HCC_2L = './raw_data/Leading_Edge/HCC-2L'
HCC_3L = './raw_data/Leading_Edge/HCC-3L'
HCC_4L = './raw_data/Leading_Edge/HCC-4L'

HCC_1T = './raw_data/Primary_Tumor/HCC-1T'
HCC_2T = './raw_data/Primary_Tumor/HCC-2T'
HCC_3T = './raw_data/Primary_Tumor/HCC-3T'
HCC_4T = './raw_data/Primary_Tumor/HCC-4T'

visium_paths = [HCC_1N, HCC_2N, HCC_3N, HCC_4N, HCC_1L, HCC_2L, HCC_3L, HCC_4L, HCC_1T, HCC_2T, HCC_3T, HCC_4T]
visium_names = ['HCC-'+ str(i + 1) + 'N-Adjacent'for i in range(4)] + \
               ['HCC-'+ str(i + 1) + 'L-Leading_Edge'for i in range(4)] + \
               ['HCC-'+ str(i + 1) + 'T-Primary_Tumor'for i in range(4)]

In [ ]:
adata_l = []
for i in range(12):
    adata = sc.read_visium(visium_paths[i])
    adata_l.append(adata)

In [ ]:
SC_genes = pd.read_csv('genes_for_nHDP.txt', header = None).values.reshape(-1).tolist()

In [ ]:
len(SC_genes)

In [ ]:
ST_genes = adata_l[0].var_names

In [ ]:
len(ST_genes)

In [ ]:
common_genes = []
for g in SC_genes:
    if g in ST_genes:
        common_genes.append(g)

In [ ]:
len(common_genes)

In [ ]:
with open('common_genes.txt', 'w') as f:
    for gene in common_genes:
        f.write(f"{gene}\n")

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(20,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        sc.pl.spatial(adata, img_key="hires", size = 0.3, ax = ax[r, c], show = False, title = visium_names[4*r + c])
        plt.tight_layout(pad=2.0)
plt.show()

In [ ]:
for i in range(12):
    adata = adata_l[i]
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

In [ ]:
patient1_adata = [adata_l[8], adata_l[4], adata_l[0]]
patient1_names = [visium_names[8], visium_names[4], visium_names[0]]

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30,10))
for r in range(3):
    adata = patient1_adata[r]
    adata.var_names_make_unique()
    sc.pl.violin(adata, 'total_counts', ax = ax[r], show = False, title = patient1_names[r])
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30,10))
for r in range(3):
    adata = patient1_adata[r]
    adata.var_names_make_unique()
    sc.pl.spatial(adata, color = 'total_counts', img_key="hires", colorbar_loc = 'bottom', size = 1.5, ax = ax[r], show = False, title = patient1_names[r])
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(20,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        adata.var_names_make_unique()
        sc.pl.violin(adata, 'n_genes_by_counts', ax = ax[r,c], show = False, title = visium_names[4*r + c])
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(25,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        adata.var_names_make_unique()
        sc.pl.violin(adata, 'total_counts', ax = ax[r,c], show = False, title = visium_names[4*r + c])
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(20,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        adata.var_names_make_unique()
        sc.pl.violin(adata, 'pct_counts_mt', ax = ax[r,c], show = False, title = visium_names[4*r + c])
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(25,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        sc.pl.spatial(adata, color = 'total_counts', img_key="hires", colorbar_loc = 'bottom', size = 1.5, ax = ax[r, c], show = False, title = visium_names[4*r + c], legend_fontsize = 8)
        plt.tight_layout(pad=2.0)
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(20,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        sc.pl.spatial(adata, color = 'n_genes_by_counts', img_key="hires", size = 1.5, ax = ax[r, c], show = False, title = visium_names[4*r + c], legend_fontsize = 8)
        plt.tight_layout(pad=2.0)
plt.show()

In [ ]:
for i in range(12):
    adata = adata_l[i]
    prepare_for_umap(adata)

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(30,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        adata.var_names_make_unique()
        sc.pl.umap(adata, color="total_counts", wspace=0.4, ax = ax[r, c], show = False, title = visium_names[4*r + c])
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(30,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        adata.var_names_make_unique()
        sc.pl.umap(adata, color="n_genes_by_counts", wspace=0.4, ax = ax[r, c], show = False, title = visium_names[4*r + c])
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(30,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        adata.var_names_make_unique()
        sc.pl.umap(adata, color="clusters", wspace=0.4, ax = ax[r, c], show = False, title = visium_names[4*r + c])
plt.show()

In [ ]:
fig, ax = plt.subplots(3,4, figsize=(20,20))
for r in range(3):
    for c in range(4):
        adata = adata_l[4*r + c]
        sc.pl.spatial(adata, color = 'clusters', img_key="hires", size = 1.5, ax = ax[r, c], show = False, title = visium_names[4*r + c], legend_fontsize = 8)
        plt.tight_layout(pad=2.0)
plt.show()

# Pooling Visiums by Class

In [ ]:
#reload all the visiums and only get X
adata_X = []
for i in range(12):
    adata = sc.read_visium(visium_paths[i])
    adata_X.append(adata.X.toarray())

In [ ]:
Adjacent_X = np.concatenate((adata_X[0], adata_X[1], adata_X[2], adata_X[3]), axis = 0)
Leading_Edge_X = np.concatenate((adata_X[4], adata_X[5], adata_X[6], adata_X[7]), axis = 0)
Primary_Tumor_X = np.concatenate((adata_X[8], adata_X[9], adata_X[10], adata_X[11]), axis = 0)

In [ ]:
Adjacent_id = []
for i in range(4):   
    Adjacent_id += [visium_names[i] for j in range(len(adata_X[i]))]

Leading_Edge_id = []
for i in range(4, 8):   
    Leading_Edge_id += [visium_names[i] for j in range(len(adata_X[i]))]
    
Primary_Tumor_id = []
for i in range(8, 12):   
    Primary_Tumor_id += [visium_names[i] for j in range(len(adata_X[i]))]

In [ ]:
Adjacent_adata = ad.AnnData(X = Adjacent_X, dtype=np.int32)
Adjacent_adata.var_names = adata_l[0].var_names
Adjacent_adata.var_names_make_unique()
Adjacent_adata.obs['Sample ID'] = Adjacent_id

Leading_Edge_adata = ad.AnnData(X = Leading_Edge_X, dtype=np.int32)
Leading_Edge_adata.var_names = adata_l[0].var_names
Leading_Edge_adata.var_names_make_unique()
Leading_Edge_adata.obs['Sample ID'] = Leading_Edge_id

Primary_Tumor_adata = ad.AnnData(X = Primary_Tumor_X, dtype=np.int32)
Primary_Tumor_adata.var_names = adata_l[0].var_names
Primary_Tumor_adata.var_names_make_unique()
Primary_Tumor_adata.obs['Sample ID'] = Primary_Tumor_id

In [ ]:
Adjacent_adata.var["mt"] = Adjacent_adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(Adjacent_adata, qc_vars=["mt"], inplace=True)

Leading_Edge_adata.var["mt"] = Leading_Edge_adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(Leading_Edge_adata, qc_vars=["mt"], inplace=True)

Primary_Tumor_adata.var["mt"] = Primary_Tumor_adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(Primary_Tumor_adata, qc_vars=["mt"], inplace=True)

In [ ]:
prepare_for_umap(Adjacent_adata)
prepare_for_umap(Leading_Edge_adata)
prepare_for_umap(Primary_Tumor_adata)

In [ ]:
sc.pl.umap(Adjacent_adata, color=["total_counts", "n_genes_by_counts", "clusters", "Sample ID"], wspace=0.4)
sc.pl.umap(Leading_Edge_adata, color=["total_counts", "n_genes_by_counts", "clusters", "Sample ID"], wspace=0.4)
sc.pl.umap(Primary_Tumor_adata, color=["total_counts", "n_genes_by_counts", "clusters", "Sample ID"], wspace=0.4)

# Pool Visiums by Patients

In [ ]:
HCC_1N = './Adjacent/HCC-1N'
HCC_2N = './Adjacent/HCC-2N'
HCC_3N = './Adjacent/HCC-3N'
HCC_4N = './Adjacent/HCC-4N'

HCC_1L = './Leading_Edge/HCC-1L'
HCC_2L = './Leading_Edge/HCC-2L'
HCC_3L = './Leading_Edge/HCC-3L'
HCC_4L = './Leading_Edge/HCC-4L'

HCC_1T = './Primary_Tumor/HCC-1T'
HCC_2T = './Primary_Tumor/HCC-2T'
HCC_3T = './Primary_Tumor/HCC-3T'
HCC_4T = './Primary_Tumor/HCC-4T'

visium_paths = [HCC_1N, HCC_2N, HCC_3N, HCC_4N, HCC_1L, HCC_2L, HCC_3L, HCC_4L, HCC_1T, HCC_2T, HCC_3T, HCC_4T]
visium_names = ['HCC-'+ str(i + 1) + 'N-Adjacent'for i in range(4)] + \
               ['HCC-'+ str(i + 1) + 'L-Leading_Edge'for i in range(4)] + \
               ['HCC-'+ str(i + 1) + 'T-Primary_Tumor'for i in range(4)]

In [ ]:
adata_l_new = []
adata_X_new = []
for i in range(12):
    adata = sc.read_visium(visium_paths[i])
    adata_l_new.append(adata)
    adata_X_new.append(adata.X.toarray())

In [ ]:
Patient1_X = np.concatenate((adata_X_new[0], adata_X_new[4], adata_X_new[8]), axis = 0)
Patient2_X = np.concatenate((adata_X_new[1], adata_X_new[5], adata_X_new[9]), axis = 0)
Patient3_X = np.concatenate((adata_X_new[2], adata_X_new[6], adata_X_new[10]), axis = 0)
Patient4_X = np.concatenate((adata_X_new[3], adata_X_new[7], adata_X_new[11]), axis = 0)

In [ ]:
Patient1_id = [visium_names[0] for j in range(len(adata_X_new[0]))] + [visium_names[4] for j in range(len(adata_X_new[4]))] + [visium_names[8] for j in range(len(adata_X_new[8]))]
Patient2_id = [visium_names[1] for j in range(len(adata_X_new[1]))] + [visium_names[5] for j in range(len(adata_X_new[5]))] + [visium_names[9] for j in range(len(adata_X_new[9]))]
Patient3_id = [visium_names[2] for j in range(len(adata_X_new[2]))] + [visium_names[6] for j in range(len(adata_X_new[6]))] + [visium_names[10] for j in range(len(adata_X_new[10]))]
Patient4_id = [visium_names[3] for j in range(len(adata_X_new[3]))] + [visium_names[7] for j in range(len(adata_X_new[7]))] + [visium_names[11] for j in range(len(adata_X_new[11]))]

In [ ]:
Patient1_adata = ad.AnnData(X = Patient1_X, dtype=np.int32)
Patient1_adata.var_names = adata_l_new[0].var_names
Patient1_adata.var_names_make_unique()
Patient1_adata.obs['Sample ID'] = Patient1_id

Patient2_adata = ad.AnnData(X = Patient2_X, dtype=np.int32)
Patient2_adata.var_names = adata_l_new[0].var_names
Patient2_adata.var_names_make_unique()
Patient2_adata.obs['Sample ID'] = Patient2_id

Patient3_adata = ad.AnnData(X = Patient3_X, dtype=np.int32)
Patient3_adata.var_names = adata_l_new[0].var_names
Patient3_adata.var_names_make_unique()
Patient3_adata.obs['Sample ID'] = Patient3_id

Patient4_adata = ad.AnnData(X = Patient4_X, dtype=np.int32)
Patient4_adata.var_names = adata_l_new[0].var_names
Patient4_adata.var_names_make_unique()
Patient4_adata.obs['Sample ID'] = Patient4_id

In [ ]:
Patient1_adata.var["mt"] = Patient1_adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(Patient1_adata, qc_vars=["mt"], inplace=True)

Patient2_adata.var["mt"] = Patient2_adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(Patient2_adata, qc_vars=["mt"], inplace=True)

Patient3_adata.var["mt"] = Patient3_adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(Patient3_adata, qc_vars=["mt"], inplace=True)

Patient4_adata.var["mt"] = Patient4_adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(Patient4_adata, qc_vars=["mt"], inplace=True)

In [ ]:
prepare_for_umap(Patient1_adata, resolution = 0.3, n_top_genes=2000, n_comps=20, batch_effects = True, df_metadata = pd.DataFrame(Patient1_id, columns = ['Sample ID']))

In [ ]:
sc.pl.umap(Patient1_adata, color=["total_counts", "n_genes_by_counts","clusters", "Sample ID"], wspace=0.4)

In [ ]:
HCC_N_cluster = Patient1_adata.obs['clusters'][0: len(adata_X_new[0])]
adata_l_new[0].obs['cluster_corr'] = HCC_N_cluster.values

In [ ]:
HCC_L_cluster = Patient1_adata.obs['clusters'][len(adata_X_new[0]): (len(adata_X_new[0]) + len(adata_X_new[4]))]
adata_l_new[4].obs['cluster_corr'] = HCC_L_cluster.values

In [ ]:
HCC_T_cluster = Patient1_adata.obs['clusters'][(len(adata_X_new[0]) + len(adata_X_new[4])): (len(adata_X_new[0]) + len(adata_X_new[4])+ len(adata_X_new[8]))]
adata_l_new[8].obs['cluster_corr'] = HCC_T_cluster.values

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30,10))
sc.pl.spatial(adata_l_new[8], color = 'cluster_corr', img_key="hires", ax = ax[0], size = 1.5, show = False, title = visium_names[8], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[4], color = 'cluster_corr', img_key="hires", ax = ax[1], size = 1.5, show = False, title = visium_names[4], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[0], color = 'cluster_corr', img_key="hires", ax = ax[2], size = 1.5, show = False, title = visium_names[0], legend_fontsize = 10)
plt.show()

In [ ]:
prepare_for_umap(Patient2_adata, resolution = 0.1, n_top_genes=2000, n_comps=20, batch_effects = True, df_metadata = pd.DataFrame(Patient2_id, columns = ['Sample ID']))

In [ ]:
sc.pl.umap(Patient2_adata, color=["total_counts", "n_genes_by_counts","clusters", "Sample ID"], wspace=0.4)

In [ ]:
HCC_N_cluster = Patient2_adata.obs['clusters'][0: len(adata_X_new[1])]
adata_l_new[1].obs['cluster_corr'] = HCC_N_cluster.values
HCC_L_cluster = Patient2_adata.obs['clusters'][len(adata_X_new[1]): (len(adata_X_new[1]) + len(adata_X_new[5]))]
adata_l_new[5].obs['cluster_corr'] = HCC_L_cluster.values
HCC_T_cluster = Patient2_adata.obs['clusters'][(len(adata_X_new[1]) + len(adata_X_new[5])): (len(adata_X_new[1]) + len(adata_X_new[5])+ len(adata_X_new[9]))]
adata_l_new[9].obs['cluster_corr'] = HCC_T_cluster.values

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30,10))
sc.pl.spatial(adata_l_new[1], color = 'cluster_corr', img_key="hires", ax = ax[0], size = 1.5, show = False, title = visium_names[1], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[5], color = 'cluster_corr', img_key="hires", ax = ax[1], size = 1.5, show = False, title = visium_names[5], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[9], color = 'cluster_corr', img_key="hires", ax = ax[2], size = 1.5, show = False, title = visium_names[9], legend_fontsize = 10)
plt.show()

In [ ]:
prepare_for_umap(Patient3_adata, resolution = 0.1, n_top_genes=2000, n_comps=20, batch_effects = True, df_metadata = pd.DataFrame(Patient3_id, columns = ['Sample ID']))

In [ ]:
sc.pl.umap(Patient3_adata, color=["total_counts", "n_genes_by_counts","clusters", "Sample ID"], wspace=0.4)

In [ ]:
HCC_N_cluster = Patient3_adata.obs['clusters'][0: len(adata_X_new[2])]
adata_l_new[2].obs['cluster_corr'] = HCC_N_cluster.values
HCC_L_cluster = Patient3_adata.obs['clusters'][len(adata_X_new[2]): (len(adata_X_new[2]) + len(adata_X_new[6]))]
adata_l_new[6].obs['cluster_corr'] = HCC_L_cluster.values
HCC_T_cluster = Patient3_adata.obs['clusters'][(len(adata_X_new[2]) + len(adata_X_new[6])): (len(adata_X_new[2]) + len(adata_X_new[6])+ len(adata_X_new[10]))]
adata_l_new[10].obs['cluster_corr'] = HCC_T_cluster.values

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30,10))
sc.pl.spatial(adata_l_new[2], color = 'cluster_corr', img_key="hires", ax = ax[0], size = 1.5, show = False, title = visium_names[2], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[6], color = 'cluster_corr', img_key="hires", ax = ax[1], size = 1.5, show = False, title = visium_names[6], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[10], color = 'cluster_corr', img_key="hires", ax = ax[2], size = 1.5, show = False, title = visium_names[10], legend_fontsize = 10)
plt.show()

In [ ]:
prepare_for_umap(Patient4_adata, resolution = 0.1, n_top_genes=2000, n_comps=20, batch_effects = True, df_metadata = pd.DataFrame(Patient4_id, columns = ['Sample ID']))

In [ ]:
sc.pl.umap(Patient4_adata, color=["total_counts", "n_genes_by_counts","clusters", "Sample ID"], wspace=0.4)

In [ ]:
HCC_N_cluster = Patient4_adata.obs['clusters'][0: len(adata_X_new[3])]
adata_l_new[3].obs['cluster_corr'] = HCC_N_cluster.values
HCC_L_cluster = Patient4_adata.obs['clusters'][len(adata_X_new[3]): (len(adata_X_new[3]) + len(adata_X_new[7]))]
adata_l_new[7].obs['cluster_corr'] = HCC_L_cluster.values
HCC_T_cluster = Patient4_adata.obs['clusters'][(len(adata_X_new[3]) + len(adata_X_new[7])): (len(adata_X_new[3]) + len(adata_X_new[7])+ len(adata_X_new[11]))]
adata_l_new[11].obs['cluster_corr'] = HCC_T_cluster.values

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(30,10))
sc.pl.spatial(adata_l_new[3], color = 'cluster_corr', img_key="hires", ax = ax[0], size = 1.5, show = False, title = visium_names[3], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[7], color = 'cluster_corr', img_key="hires", ax = ax[1], size = 1.5, show = False, title = visium_names[7], legend_fontsize = 10)
sc.pl.spatial(adata_l_new[11], color = 'cluster_corr', img_key="hires", ax = ax[2], size = 1.5, show = False, title = visium_names[11], legend_fontsize = 10)
plt.show()